# Handwritten Digit Classfication using LSTM

In this example, we are going to use the MNIST dataset to train a multi-layer feed foward neural network. MNIST is a simple computer vision dataset of handwritten digits. It has 60,000 training examles and 10,000 test examples. "It is a good database for people who want to try learning techniques and pattern recognition methods on real-world data while spending minimal efforts on preprocessing and formatting." For more details, please checkout the website [MNIST](http://yann.lecun.com/exdb/mnist/)

In [1]:
import org.apache.log4j.{Level, Logger}
import org.apache.spark.SparkContext

import com.intel.analytics.bigdl.utils._
import com.intel.analytics.bigdl.utils.{Engine, LoggerFilter, T, Table}
import com.intel.analytics.bigdl.dataset.DataSet
import com.intel.analytics.bigdl.dataset.image.{BytesToGreyImg, GreyImgNormalizer, GreyImgToBatch, GreyImgToSample}
import com.intel.analytics.bigdl.models.lenet.Utils._
import com.intel.analytics.bigdl.nn._
import com.intel.analytics.bigdl.optim._
import com.intel.analytics.bigdl.optim.{Adam, Top1Accuracy, Trigger}
import com.intel.analytics.bigdl.visualization.{TrainSummary, ValidationSummary}
import com.intel.analytics.bigdl.tensor.Tensor
import com.intel.analytics.bigdl.numeric.NumericFloat

Engine.init

## 1. Load MNIST dataset

In [2]:
import java.nio.ByteBuffer
import java.nio.file.{Files, Path, Paths}

import com.intel.analytics.bigdl.dataset.ByteRecord
import com.intel.analytics.bigdl.utils.File
import scopt.OptionParser

def load(featureFile: String, labelFile: String): Array[ByteRecord] = {
    val featureBuffer = ByteBuffer.wrap(Files.readAllBytes(Paths.get(featureFile)))
    val labelBuffer = ByteBuffer.wrap(Files.readAllBytes(Paths.get(labelFile)))
    
    val labelMagicNumber = labelBuffer.getInt()
    require(labelMagicNumber == 2049)
    val featureMagicNumber = featureBuffer.getInt()
    require(featureMagicNumber == 2051)

    val labelCount = labelBuffer.getInt()
    val featureCount = featureBuffer.getInt()
    require(labelCount == featureCount)

    val rowNum = featureBuffer.getInt()
    val colNum = featureBuffer.getInt()

    val result = new Array[ByteRecord](featureCount)
    var i = 0
    while (i < featureCount) {
      val img = new Array[Byte]((rowNum * colNum))
      var y = 0
      while (y < rowNum) {
        var x = 0
        while (x < colNum) {
          img(x + y * colNum) = featureBuffer.get()
          x += 1
        }
        y += 1
      }
      result(i) = ByteRecord(img, labelBuffer.get().toFloat + 1.0f)
      i += 1
    }

    result
}

Then we need to set paths of data. Please edit paths if they are changed.

In [3]:
val trainData = "../../datasets/mnist/train-images-idx3-ubyte"
val trainLabel = "../../datasets/mnist/train-labels-idx1-ubyte"
val validationData = "../../datasets/mnist/t10k-images-idx3-ubyte"
val validationLabel = "../../datasets/mnist/t10k-labels-idx1-ubyte"

## 2. LSTM Model Setup

This time we will use another recurrent neural network called LSTM to classify handwritten digits. You can checkout this blog to get a detailed understanding of recurrent neural networks and LSTMs in particular.

In [4]:
//Parameters
val batchSize = 64
val maxEpochs = 5

//Network Parameters
val nInput = 28 //MNIST data input (img shape: 28*28)
val nHidden = 128 // hidden layer num of features
val nClasses = 10  //MNIST total classes (0-9 digits)

Then the data set should be created and the model needs to be established.

In [5]:
val trainSet = 
    DataSet.array(load(trainData, trainLabel), sc) -> BytesToGreyImg(28, 28) -> GreyImgNormalizer(trainMean, trainStd) -> GreyImgToBatch(batchSize)
val validationSet = 
    DataSet.array(load(validationData, validationLabel), sc) -> BytesToGreyImg(28, 28) -> GreyImgNormalizer(testMean, testStd) -> GreyImgToBatch(batchSize)

In [6]:
val recurrent = Recurrent().add(LSTM(nInput, nHidden))
val rnnModel = Sequential().add(InferReshape(Array(-1, nInput), true)).add(recurrent).add(Select(2, -1)).add(Linear(nHidden, nClasses))

## 3. Optimizer Setup

In [7]:
val optimizer = Optimizer(model = rnnModel, dataset = trainSet, criterion = CrossEntropyCriterion[Float]())
optimizer.setValidation(trigger = Trigger.everyEpoch, dataset = validationSet, vMethods = Array(new Top1Accuracy))
optimizer.setOptimMethod(new Adam())
optimizer.setEndWhen(Trigger.maxEpoch(maxEpochs))

com.intel.analytics.bigdl.optim.DistriOptimizer@442350b2

The following is to create training and validation summary.

In [8]:
import java.text.SimpleDateFormat
import java.util.Calendar
val today = Calendar.getInstance
val formatDate = new SimpleDateFormat("yyyyMMdd-hhmmss")
val name = "rnn-" + formatDate.format(today.getTime()).toString()
val trainSummary = TrainSummary(logDir="/tmp/bigdl_summaries", appName=name)
trainSummary.setSummaryTrigger("Parameters", Trigger.severalIteration(50))
val valSummary = ValidationSummary(logDir="/tmp/bigdl_summaries", appName=name)
optimizer.setTrainSummary(trainSummary)
optimizer.setValidationSummary(valSummary)
printf("saving logs to %s", name)

saving logs to rnn-20170930-085704

In [9]:
// Boot training process
val trainedModel = optimizer.optimize()
print("Optimization Done.")

can't find locality partition for partition 0 Partition locations are (ArrayBuffer(172.168.2.109)) Candidate partition locations are
(0,List()).
Optimization Done.

In [10]:
val rddData = sc.parallelize(load(validationData, validationLabel), batchSize)
val transformer = BytesToGreyImg(28, 28) -> GreyImgNormalizer(testMean, testStd) -> GreyImgToSample()
val evaluationSet = transformer(rddData)
        
val result = trainedModel.evaluate(evaluationSet, Array(new Top1Accuracy[Float]), Some(batchSize))

result.foreach(r => println(s"${r._2} is ${r._1}"))

Top1Accuracy is Accuracy(correct: 9827, count: 10000, accuracy: 0.9827)


In [11]:
val predictions = trainedModel.predict(evaluationSet)
val preLables = predictions.take(8).map(_.toTensor.max(1)._2.valueAt(1)).mkString(",")
val lables = evaluationSet.take(8).map(_.label.valueAt(1)).mkString(",")
println(preLables)
println(lables)

5.0,2.0,5.0,10.0,5.0,2.0,5.0,10.0
8.0,3.0,2.0,1.0,5.0,2.0,5.0,10.0


In [12]:
import vegas._
import vegas.render.HTMLRenderer._
import vegas.DSL._

In [14]:
val loss = trainSummary.readScalar("Loss")
val top1 = valSummary.readScalar("Top1Accuracy")

val lossXY = loss.map(_ ._1).zip(loss.map(_ ._2)).toSeq

Vegas(description = "The Loss curve.", width = 700.0, height = 300.0).
  withXY(lossXY).
  encodeX("x", Quantitative, bin = Bin(maxbins = 500.0), title = "Iteration").
  encodeY("y", Quantitative, title = "Loss").
  mark(Line).
  show

val top1XY = top1.map(_ ._1).zip(top1.map(_ ._2)).toSeq
Vegas(description = "The Top1Accuracy curve.", width = 700.0, height = 300.0).
  withXY(top1XY).
  encodeX("x", Quantitative, bin = Bin(maxbins = 500.0), title = "Iteration").
  encodeY("y", Quantitative, bin = Bin(base = 0.9), title = "Top1Accuracy").
  mark(Line).
  show

<iframe id="frame-vegas-3f6c3599-2679-411a-9557-62576ea16572" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="<html>
 <head>
 <script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/d3/3.5.17/d3.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega/2.6.3/vega.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega-lite/1.2.0/vega-lite.min.js" charset="utf-8"></script>
<script src="https://vega.github.io/vega-editor/vendor/vega-embed.js" charset="utf-8"></script>
 </head>
 <body>
 <div id='vegas-3f6c3599-2679-411a-9557-62576ea16572'></div>
 <script>
 var embedSpec = {
 mode: "vega-lite",
 spec: {
 "width" : 700.0,
 "height" : 300.0,
 "mark" : "line",
 "encoding" : {
 "x" : {
 "field" : "x",
 "type" : "quantitative",
 "bin" : {
 "maxbins" : 500.0
 },
 "title" : "Iteration"
 },
 "y" : {
 "field" : "y",
 "type" : "quantitative",
 "title" : "Loss"
 }
 },
 "description" : "The Loss curve.",
 "data" : {
 "values" : [
 {
 "x" : 1,
 "y" : 2.3083999156951904
 },
 {
 "x" : 2,
 "y" : 2.3063108921051025
 },
 {
 "x" : 3,
 "y" : 2.2963109016418457
 },
 {
 "x" : 4,
 "y" : 2.2877094745635986
 },
 {
 "x" : 5,
 "y" : 2.2916173934936523
 },
 {
 "x" : 6,
 "y" : 2.2828402519226074
 },
 {
 "x" : 7,
 "y" : 2.2773406505584717
 },
 {
 "x" : 8,
 "y" : 2.2839531898498535
 },
 {
 "x" : 9,
 "y" : 2.26043701171875
 },
 {
 "x" : 10,
 "y" : 2.258389949798584
 },
 {
 "x" : 11,
 "y" : 2.2672438621520996
 },
 {
 "x" : 12,
 "y" : 2.256540060043335
 },
 {
 "x" : 13,
 "y" : 2.241849899291992
 },
 {
 "x" : 14,
 "y" : 2.2456719875335693
 },
 {
 "x" : 15,
 "y" : 2.238185405731201
 },
 {
 "x" : 16,
 "y" : 2.214872360229492
 },
 {
 "x" : 17,
 "y" : 2.2089929580688477
 },
 {
 "x" : 18,
 "y" : 2.1924405097961426
 },
 {
 "x" : 19,
 "y" : 2.202383518218994
 },
 {
 "x" : 20,
 "y" : 2.1684160232543945
 },
 {
 "x" : 21,
 "y" : 2.164116859436035
 },
 {
 "x" : 22,
 "y" : 2.1298770904541016
 },
 {
 "x" : 23,
 "y" : 2.058349609375
 },
 {
 "x" : 24,
 "y" : 2.0658864974975586
 },
 {
 "x" : 25,
 "y" : 2.058177947998047
 },
 {
 "x" : 26,
 "y" : 2.064478635787964
 },
 {
 "x" : 27,
 "y" : 2.0449578762054443
 },
 {
 "x" : 28,
 "y" : 2.0119566917419434
 },
 {
 "x" : 29,
 "y" : 1.8832545280456543
 },
 {
 "x" : 30,
 "y" : 1.9938830137252808
 },
 {
 "x" : 31,
 "y" : 1.93125319480896
 },
 {
 "x" : 32,
 "y" : 1.820232629776001
 },
 {
 "x" : 33,
 "y" : 1.9621517658233643
 },
 {
 "x" : 34,
 "y" : 1.6942787170410156
 },
 {
 "x" : 35,
 "y" : 1.9708080291748047
 },
 {
 "x" : 36,
 "y" : 1.8446320295333862
 },
 {
 "x" : 37,
 "y" : 1.4929101467132568
 },
 {
 "x" : 38,
 "y" : 1.8817977905273438
 },
 {
 "x" : 39,
 "y" : 1.670314908027649
 },
 {
 "x" : 40,
 "y" : 1.7455499172210693
 },
 {
 "x" : 41,
 "y" : 1.623070240020752
 },
 {
 "x" : 42,
 "y" : 1.7298626899719238
 },
 {
 "x" : 43,
 "y" : 1.6441961526870728
 },
 {
 "x" : 44,
 "y" : 1.5099658966064453
 },
 {
 "x" : 45,
 "y" : 1.5993285179138184
 },
 {
 "x" : 46,
 "y" : 1.3837727308273315
 },
 {
 "x" : 47,
 "y" : 1.412990927696228
 },
 {
 "x" : 48,
 "y" : 1.5541534423828125
 },
 {
 "x" : 49,
 "y" : 1.5107100009918213
 },
 {
 "x" : 50,
 "y" : 1.4232381582260132
 },
 {
 "x" : 51,
 "y" : 1.3653972148895264
 },
 {
 "x" : 52,
 "y" : 1.4627482891082764
 },
 {
 "x" : 53,
 "y" : 1.3589658737182617
 },
 {
 "x" : 54,
 "y" : 1.3150790929794312
 },
 {
 "x" : 55,
 "y" : 1.1917026042938232
 },
 {
 "x" : 56,
 "y" : 1.2639333009719849
 },
 {
 "x" : 57,
 "y" : 1.3472212553024292
 },
 {
 "x" : 58,
 "y" : 1.2253204584121704
 },
 {
 "x" : 59,
 "y" : 1.1975399255752563
 },
 {
 "x" : 60,
 "y" : 1.3078837394714355
 },
 {
 "x" : 61,
 "y" : 1.3303276300430298
 },
 {
 "x" : 62,
 "y" : 1.2957332134246826
 },
 {
 "x" : 63,
 "y" : 1.1772710084915161
 },
 {
 "x" : 64,
 "y" : 1.1208304166793823
 },
 {
 "x" : 65,
 "y" : 1.1117907762527466
 },
 {
 "x" : 66,
 "y" : 1.3332990407943726
 },
 {
 "x" : 67,
 "y" : 1.3067409992218018
 },
 {
 "x" : 68,
 "y"

<iframe id="frame-vegas-6f0b94c6-22d9-490a-bbbe-3da623f733ce" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="<html>
 <head>
 <script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/d3/3.5.17/d3.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega/2.6.3/vega.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega-lite/1.2.0/vega-lite.min.js" charset="utf-8"></script>
<script src="https://vega.github.io/vega-editor/vendor/vega-embed.js" charset="utf-8"></script>
 </head>
 <body>
 <div id='vegas-6f0b94c6-22d9-490a-bbbe-3da623f733ce'></div>
 <script>
 var embedSpec = {
 mode: "vega-lite",
 spec: {
 "width" : 700.0,
 "height" : 300.0,
 "mark" : "line",
 "encoding" : {
 "x" : {
 "field" : "x",
 "type" : "quantitative",
 "bin" : {
 "maxbins" : 500.0
 },
 "title" : "Iteration"
 },
 "y" : {
 "field" : "y",
 "type" : "quantitative",
 "bin" : {
 "base" : 0.9
 },
 "title" : "Top1Accuracy"
 }
 },
 "description" : "The Top1Accuracy curve.",
 "data" : {
 "values" : [
 {
 "x" : 938,
 "y" : 0.9623000025749207
 },
 {
 "x" : 1876,
 "y" : 0.9790999889373779
 },
 {
 "x" : 2814,
 "y" : 0.9799000024795532
 },
 {
 "x" : 3752,
 "y" : 0.9811999797821045
 },
 {
 "x" : 4690,
 "y" : 0.9825000166893005
 }
 ]
 }
}
 }
 vg.embed("#vegas-6f0b94c6-22d9-490a-bbbe-3da623f733ce", embedSpec, function(error, result) {});
 </script>

 </body>
</html>">

Finally, the Spark should be stopped.

In [15]:
sc.stop()